In [1]:
ls

assets@             pyrp/               repair.intros.md    repair.tex.raw
data/               repair.alt.md       repair.md           subp.lp
instances/          repair.bib          repair.org.md       xelatex77027.fls
main.py             repair.compact.xdv  repair.pdf
md.lp               repair.det.md       repair.stoc.md
models.ipynb        repair.html         repair.tex


In [2]:
import importlib
from collections import namedtuple as nt

In [3]:
import pandas as pd
from pyrp.util import *
from pyrp.model import repair_mip_model
import pyrp.model_single as pms
import pyrp.model_single_dp as pmsd
import pyrp.model_sg_alt as sg_alt
import importlib

## Inequality original formulation

$\begin{aligned}
q \ge h\cdot (U^\top e - D) \\
q \ge b\cdot (D - U^\top e )
\end{aligned}$

## Equality

$U^\top e - D + \delta^+ - \delta^- = 0$

## Relaxation

on $q$

### Polyak original subgrad

Only finds a primal feasible (not the feasible optimal), only gives a bound

### Volume Algorithm


Step $0 .$ We start with a vector $\bar{\pi}$ and solve (2.6) to obtain $\bar{x}$ and $\bar{z}$.
    Set $x^{0}=\bar{x}, z^{0}=\vec{z}, t=1$

Step 1. Compute $v^{t}=b-A \bar{x}$ and $\pi^{t}=\bar{\pi}+s v^{t}$ for a step size $s$ given by (2.7) Solve (2.6) with $\pi^{t},$ 

let $x^{t}$ and $z^{t}$ be the solutions obtained. Then $\bar{x}$ is updated as

$\bar{x} \leftarrow \alpha x^{t}+(1-\alpha) \bar{x}$

where $\alpha$ is a number between 0 and 1

Step 2. If $z^{t}>\bar{z}$ update $\bar{\pi}$ and $\bar{z}$ as

$\bar{\pi} \leftarrow \pi^{t}, \quad \bar{z} \leftarrow z^{t}$

Let $t \leftarrow t+1$ and go to Step 1

#### Original subgrad

relax 

$q = \max \left\{h \cdot (U ^\top e - d), b \cdot \right (d - U ^\top e) \}$

see `model_sg`

#### Alternative (equality relaxation)

relax 

$U ^\top e - d + \delta^+ - \delta^- = 0$


In [4]:
kwargs = {"subproblem_alg": 'dp',
          "mp": True,
          "scale": 15}
  
num_i = 30
num_t = 50
problem = create_instance(num_i, num_t)
scale = kwargs.get('scale', len(problem['T']))
mp = kwargs.get('mp', False)
subproblem_alg = kwargs.get('subproblem_alg', 'dp')

h, b = problem['h'], problem['p']
T = problem['T'][:scale]
I = problem['I']
D = problem['D'][:scale]
numI = len(problem['I'])

# query subproblem algorithm
if subproblem_alg == 'mip':
  # use mip
  subproblem_method = pms.single_mip
elif subproblem_alg == 'dp':
  # use dp
  subproblem_method = pmsd.single_dp
else:
  raise ValueError("unknown method for sub problem")

In [ ]:
importlib.reload(sg_alt)
importlib.reload(pmsd)

## Benchmark by MILP

In [5]:
# benchmark
model, mipsol, xsol, usol, ssol, qsol, ql, qt = repair_mip_model(problem, engine='gurobi', scale=scale)
mipsol.sum(0)

Using license file /Users/brent/licenses/gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-25
Changed value of parameter TimeLimit to 200.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
No parameters matching 'Logging' found
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1380 rows, 1365 columns and 4830 nonzeros
Model fingerprint: 0x3177b3e8
Variable types: 465 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [9e-03, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 716 rows and 377 columns
Presolve time: 0.01s
Presolved: 664 rows, 988 columns, 3058 nonzeros
Variable types: 259 continuous, 729 integer (700 binary)
Found heuristic solution: objective 325.0000000
Found heuristic solution: objective 313.0000000

Root relaxation: objective 8.742290e+00

array([[22.        ,  6.        , 25.57247773],
       [22.        ,  1.        , 21.06608208],
       [17.        , 12.        , 23.14219661],
       [15.        ,  3.        , 21.84009851],
       [12.        , 15.        , 29.45891013],
       [14.        ,  0.        , 29.7362745 ],
       [25.        ,  5.        , 35.25484114],
       [22.        ,  2.        , 30.35831974],
       [26.        ,  1.        , 28.46466944],
       [26.        ,  3.        , 24.47633237],
       [21.        ,  5.        , 21.28634025],
       [23.        ,  2.        , 19.15144664],
       [17.        ,  8.        , 19.98006037],
       [19.        ,  0.        , 17.87655229],
       [29.        ,  0.        , 16.10638738]])

In [6]:
sol, i_sol, alp, z_primal, l_b, sol_container = sg_alt.main(problem, r0=1, max_iteration=300, **kwargs)

k: 0 @obj_t: -221.10000000000002; @obj: 660.0; @lb: 0;
@stepsize: 0.32337987355110637; @norm: 330.0; @alp: 0.099


/Users/brent/Archiver/Workspace/arto/repair/pyrp/model_sg_alt.py:140: RuntimeWarning: divide by zero encountered in double_scalars
  if abs(z_primal - z_bar) / abs(z_bar) < gap:


k: 1 @obj_t: 40.0; @obj: 598.62; @lb: 0;
@stepsize: 0.3495609910037143; @norm: 299.31; @alp: 0.099
k: 2 @obj_t: 40.0; @obj: 543.3166200000001; @lb: 40.0;
@stepsize: 0.4214352276785418; @norm: 271.65831000000003; @alp: 0.099
k: 3 @obj_t: 40.0; @obj: 493.48827462; @lb: 40.0;
@stepsize: 0.5061322009340021; @norm: 246.74413731; @alp: 0.099
k: 4 @obj_t: 40.0; @obj: 448.59293543262015; @lb: 40.0;
@stepsize: 0.6051386868712415; @norm: 224.29646771631008; @alp: 0.099
k: 5 @obj_t: 40.0; @obj: 408.1422348247906; @lb: 40.0;
@stepsize: 0.7197989158329563; @norm: 204.0711174123953; @alp: 0.099
k: 6 @obj_t: 40.0; @obj: 371.69615357713644; @lb: 40.0;
@stepsize: 0.851177164718079; @norm: 185.84807678856822; @alp: 0.099
k: 7 @obj_t: 40.0; @obj: 338.8582343729999; @lb: 40.0;
@stepsize: 0.9998980615753209; @norm: 169.42911718649995; @alp: 0.099
k: 8 @obj_t: 40.0; @obj: 309.27126917007297; @lb: 40.0;
@stepsize: 1.165979621002441; @norm: 154.63563458503648; @alp: 0.099
k: 9 @obj_t: 40.0; @obj: 282.61341352

In [ ]:
mipsol.sum(0)

In [ ]:
np.array(D)

In [ ]:
sol.sum(0)

In [ ]:
i_sol

In [ ]:
sol[0]

In [ ]:
mipsol[0]

In [ ]:
dp_round_sol[0]

In [ ]:
dp_round_sol = np.zeros(shape=sol.shape)
for idx in range(num_i):
  _, round_x = pmsd.single_dp(problem, scale, l_b, idx, sol[idx][:, 0], x_target=sol[idx][:, 1])
  # _, round_x, model = pms.single_mip(problem, scale, l_b, idx, sol[idx][:, 0])
  dp_round_sol[idx, :, :] = round_x

In [ ]:
i_sol_rd = dp_round_sol.sum(0)
# i_sol_rd = sol.round(0).sum(0)
surplus = (i_sol_rd[:, 0] - D)
surplus_idx = surplus > 0
z_primal_rd = h * surplus[surplus_idx].sum(0) - b * surplus[~surplus_idx].sum(0)
i_sol_rd

In [ ]:
z_primal, z_primal_rd

In [ ]:
dp_round_sol[2], mipsol[2]